In [136]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [137]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "MED-AE"

In [138]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [139]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.107130,0.019873,-0.249654,4.565758,1.212995,0.936671,0.373433,-3.292286,-0.608011,...,-1.497216,-0.626822,-0.175596,0.310062,-0.396481,-1.039209,-0.505949,1.076142,0.747288,-0.004682
1,CHUM-002,-0.192724,-1.247029,-0.833712,4.987494,1.755802,0.080588,0.403899,-3.666713,-0.969792,...,-1.325321,-0.976352,0.337431,0.138307,-0.826634,-1.039843,0.379946,0.940582,0.739894,-0.020479
2,CHUM-006,-0.015908,-1.043045,-0.317881,6.296931,3.104916,0.227755,-0.989527,-5.192866,-1.482232,...,-1.867363,-1.959360,0.208783,-1.158109,-2.111274,-1.945736,-0.030589,2.079184,1.996498,0.509747
3,CHUM-007,-0.572432,-1.211201,0.569415,6.391412,3.528215,-0.289376,-0.501926,-5.142625,-0.647790,...,-2.122746,-1.910533,0.967195,-0.894732,-1.849706,-1.520296,0.660965,1.947969,1.768065,0.610859
4,CHUM-008,-0.031138,-1.010674,-0.776352,5.798624,2.409154,-0.690985,-0.070339,-4.241993,-0.216406,...,-1.995824,-1.745951,1.269233,-0.780660,-1.015804,-1.116197,0.515325,1.015417,1.315366,0.229303


# Up to here

Read Outcome csv

In [140]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [141]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [142]:
OD.columns[0]

'Patient_ID'

In [143]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [144]:
OD.columns[0]


'Patient_ID'

In [145]:
PET_outcome_data.columns[0]

'Patient_ID'

In [146]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,0.107130,0.019873,-0.249654,4.565758,1.212995,0.936671,0.373433,-3.292286,-0.608011,...,-1.497216,-0.626822,-0.175596,0.310062,-0.396481,-1.039209,-0.505949,1.076142,0.747288,-0.004682
1,CHUM-002,-0.192724,-1.247029,-0.833712,4.987494,1.755802,0.080588,0.403899,-3.666713,-0.969792,...,-1.325321,-0.976352,0.337431,0.138307,-0.826634,-1.039843,0.379946,0.940582,0.739894,-0.020479
2,CHUM-006,-0.015908,-1.043045,-0.317881,6.296931,3.104916,0.227755,-0.989527,-5.192866,-1.482232,...,-1.867363,-1.959360,0.208783,-1.158109,-2.111274,-1.945736,-0.030589,2.079184,1.996498,0.509747
3,CHUM-007,-0.572432,-1.211201,0.569415,6.391412,3.528215,-0.289376,-0.501926,-5.142625,-0.647790,...,-2.122746,-1.910533,0.967195,-0.894732,-1.849706,-1.520296,0.660965,1.947969,1.768065,0.610859
4,CHUM-008,-0.031138,-1.010674,-0.776352,5.798624,2.409154,-0.690985,-0.070339,-4.241993,-0.216406,...,-1.995824,-1.745951,1.269233,-0.780660,-1.015804,-1.116197,0.515325,1.015417,1.315366,0.229303


In [147]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [148]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [149]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,0.107130,0.019873,-0.249654,4.565758,1.212995,0.936671,0.373433,-3.292286,-0.608011,...,-0.175596,0.310062,-0.396481,-1.039209,-0.505949,1.076142,0.747288,-0.004682,0,1704
1,CHUM-002,-0.192724,-1.247029,-0.833712,4.987494,1.755802,0.080588,0.403899,-3.666713,-0.969792,...,0.337431,0.138307,-0.826634,-1.039843,0.379946,0.940582,0.739894,-0.020479,1,439
2,CHUM-006,-0.015908,-1.043045,-0.317881,6.296931,3.104916,0.227755,-0.989527,-5.192866,-1.482232,...,0.208783,-1.158109,-2.111274,-1.945736,-0.030589,2.079184,1.996498,0.509747,0,1186
3,CHUM-007,-0.572432,-1.211201,0.569415,6.391412,3.528215,-0.289376,-0.501926,-5.142625,-0.647790,...,0.967195,-0.894732,-1.849706,-1.520296,0.660965,1.947969,1.768065,0.610859,0,1702
4,CHUM-008,-0.031138,-1.010674,-0.776352,5.798624,2.409154,-0.690985,-0.070339,-4.241993,-0.216406,...,1.269233,-0.780660,-1.015804,-1.116197,0.515325,1.015417,1.315366,0.229303,0,1499


In [150]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)